In [1]:
from flask import Flask, request, render_template
import torch
import torchvision.transforms as transforms
from PIL import Image

app = Flask(__name__)

# Load the PyTorch model
model = torch.load('skin_disease_model.pth', map_location=torch.device('cpu'))
model.eval()  # Set the model to evaluation mode

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to the input size expected by your model
    transforms.ToTensor(),         # Convert image to a PyTorch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize (adjust based on your model training)
])

@app.route('/')
def home():
    return render_template("skin_disease_prediction.html")

@app.route('/predict', methods=['POST', 'GET'])
def predict():
    if 'file' not in request.files:
        return render_template('skin_disease_prediction.html', pred='No file uploaded.', action="Please upload an image.")

    file = request.files['file']

    if file.filename == '':
        return render_template('skin_disease_prediction.html', pred='No file selected.', action="Please upload an image.")

    try:
        # Open the uploaded image
        img = Image.open(file).convert('RGB')

        # Apply the transformations
        img = transform(img)
        img = img.unsqueeze(0)  # Add batch dimension (1, C, H, W)

        # Predict using the model
        with torch.no_grad():
            outputs = model(img)
            _, predicted = torch.max(outputs, 1)  # Get the class index with the highest score

        # Map the prediction to a disease label
        disease_mapping = {0: "Eczema", 1: "Psoriasis", 2: "Melanoma"}  # Adjust based on your model's classes
        disease = disease_mapping.get(predicted.item(), "Unknown disease")

        return render_template('skin_disease_prediction.html', 
                               pred=f'The model predicts the disease as: {disease}', 
                               action="Please consult a dermatologist for further guidance.")
    except Exception as e:
        return render_template('skin_disease_prediction.html', 
                               pred=f'Error processing image: {str(e)}', 
                               action="Please try again with a valid image.")

if __name__ == '__main__':
    app.run(debug=True)


/var/folders/4l/cj3h6cg96459x67cgqt7pzrr0000gn/T/ipykernel_57390/63995666.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('skin_disease_model.pth', ma

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'